In [2]:
from astropy.table import Table
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import json
from feadme.parser import Template

In [3]:
ingest_dir = Path("/Users/nmearl/Downloads/ZTF-DPEs-main/SDSSspectroscopy")
obj_tab = Table.read("/Users/nmearl/Downloads/ZTF-DPEs-main/object_info.cds", format='ascii.cds')
obj_tab['name'] = [x.strip() for x in obj_tab['ZTF']]
dpe_tab = Table.read("/Users/nmearl/Downloads/ZTF-DPEs-main/dpe_info.cds", format='ascii.cds')
dpe_tab['name'] = [x.strip() for x in dpe_tab['ZTF']]

In [4]:
red_dir = Path("/Users/nmearl/research/tde_agn_comparison")

In [7]:
for file_path in ingest_dir.glob("*.txt"):
    label = file_path.stem.split('_')[0]

    if label not in dpe_tab['name']:
        continue

    table = Table.read(file_path, format='ascii.basic', names=['wave', 'flux', 'flux_err'])
    redshift = obj_tab[obj_tab['name'] == label]['z'].value.data[0]

    if not redshift:
        print(f"{label} has missing redshift")
        continue

    rest_wave = table['wave'] / (1 + redshift)
    # table['rest_wave'] = rest_wave
    ha_mask = (rest_wave > 6563 - 200) & (rest_wave < 6563 + 200)

    table[ha_mask].write(red_dir / "data" / f"{label}.txt", format='ascii.basic', overwrite=True)

    with open(red_dir / "test_template.json", "r") as f:
        loaded_data = json.load(f)
        template = Template(**loaded_data)

    template.name = label
    template.redshift = redshift

    with open(red_dir / "templates" / f"{label}.json", "w") as f:
        json.dump(template.model_dump(), f)

    # print(f"Recorded redshift: {redshift}")
    # print(f"Estimated redshift: {(table['wavelength'][ha_mask][np.argmax(table['flux'][ha_mask])] - 6562.819) / 6562.819}")

    # f, ax = plt.subplots()
    #
    # # ax.errorbar(table['wavelength'], table['flux'], yerr=table['flux_err'], fmt='-')
    # if redshift:
    #     ax.plot(table['wavelength'][ha_mask], table['flux'][ha_mask])
    # else:
    #     ax.plot(table['wavelength'], table['flux'])
    # # ax.axvline(table['wavelength'][ha_mask][np.argmax(table['flux'][ha_mask])])
    # ax.axvline(6562.819 * (1 + redshift), color='red', linestyle='--')
    # f.savefig(f"{ingest_dir}/plots/{label}.png")
    # plt.close(f)


ZTF20aajbxci has missing redshift
ZTF19aaaplwg has missing redshift
ZTF19abcnoja has missing redshift
ZTF18aaltric has missing redshift
ZTF18aceypmr has missing redshift
ZTF18aasupvx has missing redshift
ZTF19aadfycj has missing redshift
ZTF19aatlxtt has missing redshift
ZTF19abcvvut has missing redshift
ZTF19abfqiaq has missing redshift
ZTF19aanffdv has missing redshift


In [30]:
from astropy.table import Table
import astropy.units as u
import astropy.uncertainty as unc
from pathlib import Path
import matplotlib.pyplot as plt

data_path = Path("/Users/nmearl/research/tde_agn_comparison/data").glob("*.txt")

for path in data_path:
    tab = Table.read(path, format='ascii.basic', names=['wave', 'flux', 'flux_err'])
    tab['flux'] = tab['flux'] * 1e-17 * u.erg / u.s / u.cm ** 2 / u.AA
    tab['flux_err'] = tab['flux_err'] * 1e-17 * u.erg / u.s / u.cm ** 2 / u.AA

    # tab['flux', 'flux_err'].pprint()
    wave = tab['wave'] * u.AA
    uflux = unc.normal(tab['flux'].quantity, std=tab['flux_err'].quantity, n_samples=100000)
    # print(uflux.pdf_median())
    uflux = uflux.to(u.mJy, u.spectral_density(wave))
    flux, flux_err = uflux.pdf_median(), uflux.pdf_std()

    tflux, tflux_err = tab['flux'].to(u.mJy, u.spectral_density(wave)), tab['flux_err'].to(u.mJy, u.spectral_density(wave))

    Table({'wave': wave, 'flux': flux, 'flux_err': flux_err}).write(path.with_suffix('.csv'), format='ascii.csv', overwrite=True)

    # f, (ax1, ax2) = plt.subplots(2,1)
    #
    # # ax.errorbar(tab['wave'], flux, yerr=flux_err, fmt='o')
    # # ax.errorbar(tab['wave'], tflux, yerr=tflux_err, fmt='o')
    # ax1.errorbar(wave, flux, yerr=flux_err, fmt='o')
    # ax1.errorbar(wave, tflux, yerr=tflux_err, fmt='o')
    # ax2.plot(flux - tflux)
    # ax2.plot(flux_err - tflux_err)
    #
    # #
    # print(flux[0], flux_err[0])
    # print(tflux[0], tflux_err[0])
    # # tab.write()
    # break